# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**План выполнения проекта:**

1. Загрузим и подготовим данные;
2. Обучим разные модели;
3. Сделаем выводы.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import torch
import time
import datetime
import random
from sklearn.metrics import f1_score
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

Загрузим и изучим данные.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.shape

(159571, 2)

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим на пропуски.

In [4]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(df)

,Total,Percent
toxic,0,0.0
text,0,0.0


Проверим на сбалансированность классов.

In [5]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Данные сильно разбалансированы, негативных твитов только 10%.
 Нужно это учесть при делении на выборки.

### Подготовка к обучению

Загружаем датасет, берём определенное количество твитов случайным
 образом с учетом баланса классов, возвращаем отдельно сами твиты и метки.

In [7]:
def get_balanced_dataframe(count=None, link_csv='/datasets/toxic_comments.csv'):
    df1 = pd.read_csv(link_csv)
    print('Размер изначального датасета', df1.shape)
    if count:
        df_pos = df1[df1['toxic']==0].sample(int(count/2))
        df_neg = df1[df1['toxic']==1].sample(int(count/2))
        df1 = pd.concat([df_pos, df_neg])
        print('Размер подвыборки', df1.shape)
    print('10 случайных примеров из датасета', df1.sample(10))
    return df1

In [8]:
def get_text_labels(df):
    sentences = df['text'].values
    labels = df['toxic'].values
    return sentences, labels

Чтобы долго не обучать модель берем 2000 сбалансированных твитов.
 Балансируем только тренировочную выборку. Тест - должен быть чистый, не балансированный.

In [9]:
balanced_df = get_balanced_dataframe(count=1600)
sentences_train, labels_train = get_text_labels(balanced_df)

Размер изначального датасета (159571, 2)
Размер подвыборки (1600, 2)
10 случайных примеров из датасета                                                      text  toxic
17174   "\n\nJanuary 24, 2007\nYour change  was determ...      0
52565         You obviously never attended college, moron      1
1651    You won't find any creditable citations becaus...      1
139350  Chavez sure isnt a hero. He's classic scum. An...      1
129445  "\n\nThere is a country called America. There ...      0
157647  It is not off board dispute, because legally W...      0
120670           Please check the bottom of my talk page.      0
32988   "\n\nJosif Stalin IS one of the three worst cr...      0
102782  I think that if you are going to use ther exac...      0
140888  Aquabuddha \n\nWhy is there no mention of his ...      0


Вычтем 1600 тех которые взяли для обучения из общей выборки и возьмем еще 400 случайных не сбалансированных. Для наглядности посмотрим на изначальный размер датасета и после вычитания.

In [10]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.shape

(159571, 2)

In [11]:
unbalanced_df = df[(~df.index.isin(balanced_df.index))]
unbalanced_df.shape

(157971, 2)

In [12]:
df_valid = unbalanced_df.sample(400)
df_valid['toxic'].value_counts()

0    367
1     33
Name: toxic, dtype: int64

In [13]:
sentences_valid, labels_valid = get_text_labels(df_valid)

Получаем токенайзер. Приводим к нижнему регистру. И проверяем его работу.

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
sentence_number = 0
print('Original: ', sentences_train[sentence_number])
print()
print('Tokenized: ', tokenizer.tokenize(sentences_train[sentence_number]))
print()
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences_train[sentence_number])))

Original:  "

 Asking users to contact you privately 

Dear editor at 84.75.133.108. You are currently (as you have done in the past) contacting users, in particular administrators, asking them to e-mail you privately about something. I do not know what you wish to talk about but I can assure you that there is probably a better way to accomplish whatever it is that you wish to accomplish. In particular, if you have a legal concern about your personal information on any Wikipedia article, please contact the OTRS team as per Wikipedia:BLPEDIT#Legal_issues. If you are being harassed in some manner, please see Wikipedia:Harassment for advice. If it's a research project, read Wikipedia:Research recruitment. If it's a copyright problem, please follow the directions at Wikipedia:Copyright_violations#Information_for_copyright_owners. I can't really think of too many other problems that are better handled off-wiki and privately. Please be aware of our guidelines against ""canvassing"", which is

Токенизация всех предложений в датасете. С помощью encode_plus токенизируем, приводим к определнной длине, получаем маску "внимания" (attention_mask).

In [15]:
def get_input_attention_mask_labels(sentences, labels):
    input_ids, attention_masks = [], []

    for sent in sentences:
        encoder_dict = tokenizer.encode_plus(
        sent, # предложение для токенизации
        add_special_tokens=True, # добавляем токен в начало и конец предложения
        max_length=64, # ограничиваем максимальную длину выходных токенизированных строк
        pad_to_max_length=True,
        return_attention_mask=True, # возвращаем attention_mask
        return_tensors='pt' # получить данные в виде тензоров pytorch
      )

        input_ids.append(encoder_dict['input_ids'])
        attention_masks.append(encoder_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)
    return input_ids, attention_masks, labels

Конвертируем списки в отдельные тензоры PyTorch.

In [16]:
input_ids_train, attention_masks_train, labels_train = get_input_attention_mask_labels(sentences_train, labels_train)

input_ids_valid, attention_masks_valid, labels_valid = get_input_attention_mask_labels(sentences_valid, labels_valid)

Разбиваем данные на обучающую и валидационную выборки.

In [17]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_valid, attention_masks_valid, labels_valid)


Разделим данные на батчи.

In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

Создаем модель нейронной сети. Берём предобученную 12 слойную
модель BERT без регистра "bert-base-uncased".

In [19]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # используем 12 слойную модель BERT без регистра
    num_labels=2, # количество выходных слоев для бинарной классификации
    output_attentions=False, # не возвращаем веса для attention слоев
    output_hidden_states=False # не возвращаем веса для скрытых слоев
)

#model.cuda()

Создаем оптимизатор для регулирования весов и скорости обучения.
Задаем количество эпох и планировщик Learning rate.

In [20]:
optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  eps=1e-8
                  )

from transformers import get_linear_schedule_with_warmup

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

Методы расчета времени, точности и f1.

In [21]:
def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)

Теперь можно переходить к самому обучению.

## Обучение

Для сравнения проверим две модели нейронку BERT и старый добрый подход TF_IDF
с моделью CatBoostClassifier.

### BERT

Обучение модели производилось в Google Colab ссылка
на ноутбук с выводами
https://colab.research.google.com/drive/1Rfti84o9ujvP_t8ct8Aa-WS3Kw7PZT-J?usp=sharing ниже
 так же продублирован вывод в результате работы кода.

In [22]:
def train_step(device, model, train_dataloader, optimizer, scheduler):
    t0 = time.time()
    total_train_loss = 0
    # переводим модель в режим тренировки
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(' Batch {:>5,} of {:>5}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # извлекаем все компоненты из полученного батча
        b_input_ids, b_inpit_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
        # очищаем ранее посчитанные градиенты
        model.zero_grad()
        # выполняем прямой проход по данным
        loss, logits = model(b_input_ids, token_type_ids=None, attention_mask=b_inpit_mask, labels=b_labels)
        # накапливаем тренировочную функцию потерь по всем батчам
        total_train_loss += loss.item()
        # выполняем обратное распространение ошибки, чтобы посчитать градиент
        loss.backward()
        # ограничиваем максимальный размер градиента до 1.0
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
        # обновляем параметры модели используя рассчитанные градиенты с помощью выбранного оптимизатора и 
        # текущего learning rate
        optimizer.step()
        # обновляем learning rate
        scheduler.step()
    # считаем среднее значение функции потерь
    avg_train_loss = total_train_loss / len(train_dataloader)

    training_time = format_time(time.time() - t0)
    print(" Average training loss: {0:.2f}".format(avg_train_loss))
    print(" Training epoch took: {:}".format(training_time))
    return avg_train_loss, training_time

In [23]:
def validation_step(device, model, validation_dataloader):
    # переводим модель в режим evaluation 
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1_score = 0
    for batch in validation_dataloader:
         # извлекаем все компоненты из полученного батча
        b_input_ids, b_input_mask, b_labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)
         # говорим что нам не нужен вычислительный граф для подсчета градиентов
        with torch.no_grad():
            # прямой проход по нейросети и получение выходных значений
            (loss, logits) = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # накапливаем значения для функции потерь для валидации
        total_eval_loss += loss.item()
        # переносим значения с GPU на CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # считаем точьность для отдельного батча с текстами и накапливаем значение
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        total_eval_f1_score += flat_f1(logits, label_ids)
    
    avg_val_f1_score = total_eval_f1_score / len(validation_dataloader)
    print(" F1_score: {0:.2f}".format(avg_val_f1_score))

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print(" Accuracy: {0:.2f}".format(avg_val_accuracy))
    # считаем среднюю функцию потерь для всех батчей
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print(" Validation Loss: {0:.2f}".format(avg_val_loss))

    return avg_val_loss, avg_val_accuracy, avg_val_f1_score

Проверяем есть ли в системе GPU с CUDA если есть используем его, иначе работает на CPU.

In [24]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        print('We will use the GPU:', torch.cuda.get_device_name(0))
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device

In [ ]:
training_stats = []
total_t0 = time.time()
device = get_device()
for epoch_i in range(0, epochs):
    avg_train_loss, training_time = train_step(device,
                                               model,
                                               train_dataloader,
                                               optimizer,
                                               scheduler)
    avg_val_loss, avg_val_accuracy, avg_val_f1_score = validation_step(device,
                                                     model,
                                                     validation_dataloader)
    training_stats.append(
        {
            'Epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Validation Loss': avg_val_loss,
            'Validation Accur.': avg_val_accuracy,
            'F1 score': avg_val_f1_score,
            'Training Time': training_time
        }
    )
print("Training complete! Total training took {:} (hh:mm:ss)".format(format_time((time.time() - total_t0))))

### Вывод BERT:

There are 1 GPU(s) available.  
We will use the GPU: Tesla T4  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: torch.nn.utils.clip_grad_norm is now   deprecated in favor of torch.nn.utils.clip_grad_norm_.  
 Batch    40 of    50. Elapsed: 0:00:15.  
 Average training loss: 0.42  
 Training epoch took: 0:00:19  
 F1_score: 0.65  
 Accuracy: 0.94  
 Validation Loss: 0.16  
 Batch    40 of    50. Elapsed: 0:00:15.  
 Average training loss: 0.18  
 Training epoch took: 0:00:19  
 F1_score: 0.66  
 Accuracy: 0.93  
 Validation Loss: 0.21  
 Batch    40 of    50. Elapsed: 0:00:15.  
 Average training loss: 0.10  
 Training epoch took: 0:00:19  
 F1_score: 0.67  
 Accuracy: 0.93  
 Validation Loss: 0.21  
 Batch    40 of    50. Elapsed: 0:00:15.  
 Average training loss: 0.06  
 Training epoch took: 0:00:18  
 F1_score: 0.64  
 Accuracy: 0.92  
 Validation Loss: 0.24  
Training complete! Total training took 0:01:22 (hh:mm:ss)    

Если проверять на несбалансированной выборке то F1_score всего 0.64, ниже требуемых 0,75, поэтому для интереса я сделал еще по всей выборке (159_571 твитов) без балансировки https://colab.research.google.com/drive/1MhTD2-di5ilMfBh017_TAOR8JLIxSjlY?usp=sharing ниже так же продублирована часть вывода в результате работы кода.

 Batch 3,680 of  3990. Elapsed: 0:40:28.  
 Batch 3,720 of  3990. Elapsed: 0:40:54.  
 Batch 3,760 of  3990. Elapsed: 0:41:21.  
 Batch 3,800 of  3990. Elapsed: 0:41:47.  
 Batch 3,840 of  3990. Elapsed: 0:42:13.  
 Batch 3,880 of  3990. Elapsed: 0:42:40.  
 Batch 3,920 of  3990. Elapsed: 0:43:06.  
 Batch 3,960 of  3990. Elapsed: 0:43:33.  
 Average training loss: 0.02  
 Training epoch took: 0:43:52  
 F1_score: 0.77  
 Accuracy: 0.96  
 Validation Loss: 0.22  
Training complete! Total training took 3:10:34 (hh:mm:ss)  

Точность F1_score: 0.77 при общем времени на 4 эпохи 3часа 10минут 34секунды по всей выборке в 159_571 твитов.

### TF_IDF

Загрузим все необходимы для обработки текста библиотеки.

In [42]:
import re
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
m = Mystem()
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
def preprocessing_text(row):
    text = row['text']
    text_re = ' '.join(re.sub("[^a-zA-Z ]", " ", text).lower().split())
    lemmas = m.lemmatize(text_re)
    row['lemma_text'] = ' '.join([token for token in lemmas
                       if token not in ['\n',' \n'] and
                       len(token) > 1 and
                       token not in stopwords and
                       token.strip() not in punctuation])

    return row

Так же как и для модели BERT возьмем 1600 сбалансированных твитов.

In [45]:
def get_df_train_valid(df, count=2000, size=0.8):
    size_train = int(count*size)
    size_valid = count - size_train
    df_train = get_balanced_dataframe(count=size_train)
    print('Всего негативных комментариев в тренировочной', df_train.toxic.sum())
    unbalanced_df = df[(~df.index.isin(df_train.index))]
    df_valid = unbalanced_df.sample(size_valid)
    return df_train, df_valid

Очистим и лемматизируем.

In [46]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df_train, df_valid = get_df_train_valid(df, count=2000, size=0.8)
df_train = df_train.apply(preprocessing_text, axis=1)
df_valid = df_valid.apply(preprocessing_text, axis=1)

Размер изначального датасета (159571, 2)
Размер подвыборки (1600, 2)
10 случайных примеров из датасета                                                      text  toxic
122112  "\nOK. I was about to say I had no clue what w...      0
99195   Suck my dikkkkk ==\nSuck my dikkkkk Gogo Gogo ...      1
33935   "WARNING: MrZaius has a hard-on for the term "...      1
110749  Doesn't matter  you are still using it for pur...      0
114385  "\n\n Thanks \n\n  The Random Acts of Kindness...      0
76164   Bloody asshole \n\nYou are a bloody asshole an...      1
82182       AM A GAY FAG THAT FUCKS MY MOM IN HER FAT ASS      1
110078  The inhabitants of Istanbul call the city Ista...      0
136467  Go take a long walk off a short pier. \n\nSee ...      1
38033                Go fuck yourself you lifeless pedant      1
Всего негативных комментариев в тренировочной 800


In [39]:
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

Обучим модель CatBoostClassifier на 2_000 примеров.

In [41]:
def get_predict_by_tf_idf(df_train, df_valid):
    target_train = df_train['toxic']
    features_train = df_train.drop(['toxic','text'], axis=1)
    
    target_valid = df_valid['toxic']
    features_valid = df_valid.drop(['toxic','text'], axis=1)


    corpus_train = features_train['lemma_text'].values.astype('U')
    corpus_valid = features_valid['lemma_text'].values.astype('U')

    count_tf_idf_train = TfidfVectorizer(stop_words=stopwords)
    tf_idf_train = count_tf_idf_train.fit_transform(corpus_train)

    model = CatBoostClassifier(verbose=100)
    model.fit(tf_idf_train, target_train)

    tf_idf_test = count_tf_idf_train.transform(corpus_valid)

    pred_valid = model.predict(tf_idf_test)
    print(f1_score(target_valid, pred_valid))
    
get_predict_by_tf_idf(df_train, df_valid)

Learning rate set to 0.019138
0:	learn: 0.6874258	total: 118ms	remaining: 1m 57s
100:	learn: 0.5205111	total: 14.5s	remaining: 2m 8s
200:	learn: 0.4669611	total: 29.4s	remaining: 1m 56s
300:	learn: 0.4283921	total: 44.3s	remaining: 1m 42s
400:	learn: 0.3912648	total: 59.1s	remaining: 1m 28s
500:	learn: 0.3507106	total: 1m 13s	remaining: 1m 13s
600:	learn: 0.3173272	total: 1m 28s	remaining: 58.5s
700:	learn: 0.2909398	total: 1m 42s	remaining: 43.6s
800:	learn: 0.2695489	total: 1m 56s	remaining: 29s
900:	learn: 0.2510792	total: 2m 11s	remaining: 14.4s
999:	learn: 0.2352781	total: 2m 25s	remaining: 0us
0.5416666666666666


### Вывод TF_IDF (при 2_000 примеров):

Learning rate set to 0.019138   
0:	learn: 0.6874258	total: 118ms	remaining: 1m 57s  
100:	learn: 0.5205111	total: 14.5s	remaining: 2m 8s  
200:	learn: 0.4669611	total: 29.4s	remaining: 1m 56s  
300:	learn: 0.4283921	total: 44.3s	remaining: 1m 42s  
400:	learn: 0.3912648	total: 59.1s	remaining: 1m 28s  
500:	learn: 0.3507106	total: 1m 13s	remaining: 1m 13s  
600:	learn: 0.3173272	total: 1m 28s	remaining: 58.5s  
700:	learn: 0.2909398	total: 1m 42s	remaining: 43.6s  
800:	learn: 0.2695489	total: 1m 56s	remaining: 29s  
900:	learn: 0.2510792	total: 2m 11s	remaining: 14.4s  
999:	learn: 0.2352781	total: 2m 25s	remaining: 0us  
0.5416666666666666   

F1_score = 0.5417

Возьмем 10_000 примеров.

In [47]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df_train, df_valid = get_df_train_valid(df, count=10000, size=0.8)
df_train = df_train.apply(preprocessing_text, axis=1)
df_valid = df_valid.apply(preprocessing_text, axis=1)

Размер изначального датасета (159571, 2)
Размер подвыборки (8000, 2)
10 случайных примеров из датасета                                                      text  toxic
70374   "\nNational Foundation Day is an ""ancient his...      0
35109   butt butt butt butt butt butt butt butt butt b...      1
115364  template:user atheist and zen buddhist \nappar...      0
121467  "\n\n Always complaining... \n\nYou know, nigg...      1
84709   Peacemaker, how evil you must be when you now ...      1
81651   - Bridgnorth.jpg|left|thumb|]]\n\nBridgnorth M...      0
49745   "\n\n Listen up fagtard \n\nHow can the fact t...      1
45746   "\n\n User: Jhoney123 \n\nI feel Jhoney123 sho...      1
37756   Who think you are ????  You the worst offender...      1
56289      I'm a fat pig.  Does that belong on Wikipedia?      1
Всего негативных комментариев в тренировочной 4000


Обучим модель CatBoostClassifier на 10_000 примеров.

In [48]:
get_predict_by_tf_idf(df_train, df_valid)

Learning rate set to 0.030179
0:	learn: 0.6813042	total: 692ms	remaining: 11m 31s
100:	learn: 0.4658948	total: 53.6s	remaining: 7m 57s
200:	learn: 0.4130504	total: 1m 45s	remaining: 7m
300:	learn: 0.3782224	total: 2m 37s	remaining: 6m 6s
400:	learn: 0.3444523	total: 3m 29s	remaining: 5m 13s
500:	learn: 0.3184733	total: 4m 21s	remaining: 4m 20s
600:	learn: 0.2968444	total: 5m 15s	remaining: 3m 29s
700:	learn: 0.2787255	total: 6m 7s	remaining: 2m 36s
800:	learn: 0.2635980	total: 6m 59s	remaining: 1m 44s
900:	learn: 0.2516716	total: 7m 51s	remaining: 51.8s
999:	learn: 0.2401656	total: 8m 43s	remaining: 0us
0.6584766584766585


### Вывод TF_IDF (при 10_000 примеров):

Learning rate set to 0.030179  
0:	learn: 0.6813042	total: 692ms	remaining: 11m 31s  
100:	learn: 0.4658948	total: 53.6s	remaining: 7m 57s  
200:	learn: 0.4130504	total: 1m 45s	remaining: 7m  
300:	learn: 0.3782224	total: 2m 37s	remaining: 6m 6s  
400:	learn: 0.3444523	total: 3m 29s	remaining: 5m 13s  
500:	learn: 0.3184733	total: 4m 21s	remaining: 4m 20s  
600:	learn: 0.2968444	total: 5m 15s	remaining: 3m 29s  
700:	learn: 0.2787255	total: 6m 7s	remaining: 2m 36s  
800:	learn: 0.2635980	total: 6m 59s	remaining: 1m 44s  
900:	learn: 0.2516716	total: 7m 51s	remaining: 51.8s  
999:	learn: 0.2401656	total: 8m 43s	remaining: 0us  
0.6584766584766585    

F1_score = 0.6585

Так же как и с BERT сделана методика
TF_IDF c CatBoostClassifier по всей (несбалансированной)
 выборке, но уже в DataShpere аналог Colab, но от Яндекса.
 Ниже вывод в результате выполнения и время(32 ядра и 4 Tesla V100):

Learning rate set to 0.081698  
0:	learn: 0.6113123	total: 976ms	remaining: 16m 15s  
100:	learn: 0.1779025	total: 1m 6s	remaining: 9m 52s  
200:	learn: 0.1544423	total: 2m 16s	remaining: 9m 2s  
300:	learn: 0.1418005	total: 3m 28s	remaining: 8m 3s  
400:	learn: 0.1331876	total: 4m 42s	remaining: 7m 2s  
500:	learn: 0.1261722	total: 5m 57s	remaining: 5m 55s  
600:	learn: 0.1208737	total: 7m 12s	remaining: 4m 47s  
700:	learn: 0.1166288	total: 8m 27s	remaining: 3m 36s  
800:	learn: 0.1127415	total: 9m 43s	remaining: 2m 25s  
900:	learn: 0.1095175	total: 10m 59s	remaining: 1m 12s  
999:	learn: 0.1062234	total: 12m 15s	remaining: 0us  
0.7559168925022584  

## Выводы


| Модель                   | F1-score   | Примеров train |  Примеров valid | max_length_text | batch_size |
| :------------------------|-----------:|---------------:|----------------:|----------------:|-----------:|
| bert-base-uncased 2000   |     0.6400 |          1_600 |             400 |             64  |        32  | 
| bert-base-uncased 10000  |     0.6200 |          8_000 |           2_000 |             64  |        32  |
| bert-base-uncased all    | **0.7700** |        127_657 |          31_914 |             64  |        32  |
| CatBoostClassifier 2000  |     0.5417 |          1_600 |             400 |              -  |         -  |
| CatBoostClassifier 10000 | **0.6585** |          8_000 |           2_000 |              -  |         -  |
| CatBoostClassifier all   |     0.7559 |        127_657 |          31_914 |              -  |         -  |

В этой работе у нас был датасет из 159_571 твитов, из них 16_225 (10%)
негативные и 143_346 позитивные. Из несбалансированной выборки сделали
 сбалансированную, взяли всего 1600 твитов по 800 позитивных и негативных.
 Обучал на 400 случайных.
  
Разделили выборку на тренировочную 80% и валидационную 20%. Провели
токенизацию, взяли максимальную длину в 64 токена. Разделили данные
на батчи размером 32. Взяли предобученную 12 слойную модель BERT без
 регистра "bert-base-uncased". Обучали 4 эпохи как рекомендуют создатели BERT.
  
В результате F1_score последней эпохи равно 0.64, за полторы минуты (01:22).
Что показывает не очень хорошую точность. Датасет из 10_000 дал похожие
 результаты (поэтому его я сюда не выводил, обучал в колабе).
  
Так же был применен подход TF_IDF, взято 2_000, 10_000 сбалансированных
примеров, лематизированно, а затем передано в CatBoostClassifier.
На 2_000 получил F1_score=0.5417, что ниже заданной метрики (0.75),
но обучалась модель около 10 минут, что дольше чем BERT и результат похуже.
 На 10_000 примеров CatBoost обучался десятки минут, с точностью F1_score=0.6585,
  что уже близко к BERT.
  
Если учить вообще без изначальной балансировки и по всем параметрам
то точность BERT 0.77, а CatBoostClassifier 0,76 и по скорости 3,5 часа
 и десятки минут, но опять же на разных мощностях, поэтому время сравнивать
 не совсем корректно. Нельзя сказать однозначно кто лидирует, модель на основе
  нейронной 12 слойной сети BERT без регистра "bert-base-uncased" или
   CatBoostClassifier, модели показали одинаково плохой результат, хоть
   он и дотягивает до требуемых 0,75, но учитывая разбаланс(90% и 10%)
   если предсказывать всем твитам положительную метку у нас будет точность
   90%. Но бустинг CatBoostClassifier показал сопоставимый по точности
   результат с нейронкой BERT.

Из плюсов использования CatBoostClassifier - его гораздо проще конфигурировать,
он показал неплохой результат "из коробки" даже без настройки, и намного
 удобнее готовить данные для обучения.